In [1]:
import findspark
findspark.init() # this must be executed before the below import

In [2]:
from pyspark.sql import SparkSession
spark = SparkSession \
    .builder \
    .master("local") \
    .appName("Python Spark SQL Execution") \
    .config("spark.executor.memory", "8g") \
    .config("spark.driver.memory","8g") \
    .config("spark.memory.offHeap.enabled",True) \
    .config("spark.memory.offHeap.size","8g") \
    .getOrCreate()

In [3]:
# none of these can affect "second time faster"
# .config("spark.sql.optimizer.metadataOnly", False)
# .config("spark.shuffle.service.index.cache.size", "0m")
# .config("spark.files.useFetchCache", False)

In [4]:
import numpy as np
import time
import rtree
from rtree import index

In [5]:
from NORAPartitionTree import *

In [6]:
def find_overlap_parquets(query, partition_index):
    '''
    find out all the overlap partition ids
    '''
    query_lower = [qr[0] for qr in query]
    query_upper = [qr[1] for qr in query]
    query_border = tuple(query_lower + query_upper)
    overlap_pids = list(partition_index.intersection(query_border))
    
    return overlap_pids

In [7]:
def transform_query_to_sql(query, used_dims, column_name_dict, hdfs_path, querytype = 0, pids = None):
    sql = ''
    for i, dim in enumerate(used_dims):
        #if query[i][0] != -1:
        sql += column_name_dict[dim] + '>' + str(query[i]) + ' and '
        #if query[i][1] != -1:
        sql += column_name_dict[dim] + '<' + str(query[len(used_dims)+i]) + ' and '
    sql = sql[0:-4] # remove the last 'and '
    print("pids:",pids)
    if pids is not None and len(pids) != 0:
        pids = str(set(pids)).replace(" ", "") # '{1,2,3}'
        hdfs_path = hdfs_path + '/partition_' + pids + ".parquet"
    
    if querytype == 0:
        sql = "SELECT * FROM parquet.`" + hdfs_path + "`WHERE " + sql
    elif querytype == 1:
        sql = "SELECT COUNT(*) FROM parquet.`" + hdfs_path + "`WHERE " + sql
    elif querytype == 2:
        sql = "SELECT variance(_c0) FROM parquet.`" + hdfs_path + "`WHERE " + sql
            
    #else:
        #pids = str(set(pids)).replace(" ", "") # '{1,2,3}'
        #sql = "SELECT * FROM parquet.`" + hdfs_path + 'partition_' + pids + ".parquet` WHERE " + sql
        #sql = "SELECT COUNT(*) FROM parquet.`" + hdfs_path + 'partition_' + pids + ".parquet` WHERE " + sql
        #sql = "SELECT variance(_c0) FROM parquet.`" + hdfs_path + 'partition_' + pids + ".parquet` WHERE " + sql
    return sql

In [8]:
def query_with_parquets(query, used_dims, column_name_dict, hdfs_path, querytype = 0, partition_tree = None, print_execution_time = False):

    start_time = time.time()
    
    sql = None    
    
    if partition_tree == None:
        sql = transform_query_to_sql(query, used_dims, column_name_dict, hdfs_path, querytype)
    else:
        pids = partition_tree.query_single(query) # find_overlap_parquets(query, rtree_idx)
        sql = transform_query_to_sql(query, used_dims, column_name_dict, hdfs_path, querytype, pids)
        #print(sql)
    
    #print("generated sql:", sql)
    end_time_1 = time.time()
    
    query_result = spark.sql(sql).collect()
#     query_result = spark.sql(sql) # lazy execution
#     query_time = spark.time(spark.sql(sql).collect())  # there is no .time in pyspark
    
    end_time_2 = time.time()
    
#     print("result size:", len(query_result))
#     print("result content:", query_result)
    
    query_translation_time = end_time_1 - start_time
    query_execution_time = end_time_2 - end_time_1
    #print('query execution time: ', query_execution_time)
    
    if print_execution_time:
        print('query translation time: ', query_translation_time)
        print('query execution time: ', query_execution_time)
    
    #return (query_result, query_translation_time, query_execution_time) # this takes too much memory
    return (query_translation_time, query_execution_time, len(query_result))

In [9]:
def load_query(path):
    query_set = np.genfromtxt(path, delimiter=' ')
    #query_set = query_set.reshape(len(query_set),-1,2)
    return query_set

def kdnode_2_border(kdnode):
    lower = [domain[0] for domain in kdnode[0]]
    upper = [domain[1] for domain in kdnode[0]]
    border = tuple(lower + upper) # non interleave
    return border

def load_partitions_from_file(path):
    '''
    the loaded stretched_kdnodes: [num_dims, l1,l2,...,ln, u1,u2,...,un, size, id, pid, left_child,id, right_child_id]
    '''
    stretched_kdnodes = np.genfromtxt(path, delimiter=',')
    num_dims = int(stretched_kdnodes[0,0])
    kdnodes = []
    for i in range(len(stretched_kdnodes)):
        domains = [ [stretched_kdnodes[i,k+1],stretched_kdnodes[i,1+num_dims+k]] for k in range(num_dims) ]
        row = [domains]
        row.append(stretched_kdnodes[i,2*num_dims+1])
        # to be compatible with qd-tree's partition, that do not have the last 4 attributes
        if len(stretched_kdnodes[i]) > 2*num_dims+2:
            row.append(stretched_kdnodes[i,-4])
            row.append(stretched_kdnodes[i,-3])
            row.append(stretched_kdnodes[i,-2])
            row.append(stretched_kdnodes[i,-1])
        kdnodes.append(row)
    return kdnodes

# def prepare_partition_index(partition_path):
#     partitions = load_partitions_from_file(partition_path)

#     p = index.Property()
#     p.leaf_capacity = 32
#     p.index_capacity = 32
#     p.NearMinimumOverlaoFactor = 16
#     p.fill_factor = 0.8
#     p.overwrite = True
#     pidx = index.Index(properties = p)

#     partition_index = index.Index(properties = p)
#     for i in range(len(partitions)):
#         partition_index.insert(i, kdnode_2_border(partitions[i]))
    
#     return partition_index

def batch_query(queryset, used_dims, column_name_dict, hdfs_path, querytype = 0, partition_path = ""):
    
#     rtree_idx = None
#     if use_rtree_idx:
#         rtree_idx = prepare_partition_index(partition_path)
    
    partition_tree = PartitionTree(len(used_dims)) # newly added
    partition_tree.load_tree(partition_path)
    
    start_time = time.time()
    
    # add statistics result
    results = []
    count = 0
    for i in range(0, len(queryset)):
        result = query_with_parquets(queryset[i], used_dims, column_name_dict, hdfs_path, querytype, partition_tree)
        print('finish query', count)
        count += 1
        results.append(result)
        #print("query:",queryset[i])
#         if i == 0:
#             break # just analysis top k queries
    end_time = time.time()
    
    result_size = 0
    for result in results:
        result_size += result[2]
    avg_result_size = int(result_size // len(queryset))
    
    print('total query response time: ', end_time - start_time)
    print('average query response time: ', (end_time - start_time) / len(queryset))
    print('average result size: ', avg_result_size)

In [10]:
# # = = = Configuration (COMP Cloud Ubuntu) = = =

# scale_factor = 100
# # query_base_path = '/home/cloudray/NORA_Query/'
# query_base_path = '/home/ubuntu/Queryset/'

# distribution_path = query_base_path + 'distribution_' + str(scale_factor) + '.csv'
# random_path = query_base_path + 'random_' + str(scale_factor) + '.csv'

# distribution_query = load_query(distribution_path)
# random_query = load_query(random_path)

# training_set_percentage = 0.5
# Td = int(len(distribution_query) * training_set_percentage)
# Tr = int(len(random_query) * training_set_percentage)

# training_set = np.concatenate((distribution_query[0:Td], random_query[0:Tr]), axis=0)
# testing_set = np.concatenate((distribution_query[Td:], random_query[Tr:]), axis=0)

# used_dims = [1,2]
# num_dims = 16
# column_names = ['_c'+str(i) for i in range(num_dims)]
# column_name_dict = {}
# for i in range(num_dims):
#     column_name_dict[i] = column_names[i]

# # hdfs_path_nora = 'hdfs://localhost:9000/user/cloudray/NORA/merged/'
# # hdfs_path_qdtree = 'hdfs://localhost:9000/user/cloudray/QdTree/merged/'

# hdfs_path_nora = 'hdfs://10.88.88.103:9000/user/cloudray/NORA/scale100/merged/'
# hdfs_path_qdtree = 'hdfs://10.88.88.103:9000/user/cloudray/QdTree/scale100/merged/'
# hdfs_path_kdtree = 'hdfs://10.88.88.103:9000/user/cloudray/KDTree/scale100/merged/'

# # partition_base_path = '/home/ubuntu/PartitionLayout/'
# # nora_partition = partition_base_path + 'nora_partitions_' + str(scale_factor)

In [11]:
# = = = Configuration (UBDA Cloud Centos) = = =

scale_factor = 10
# query_base_path = '/home/centos/Queryset/'

# distribution_path = query_base_path + 'distribution_' + str(scale_factor) + '.csv'
# random_path = query_base_path + 'random_' + str(scale_factor) + '.csv'

# distribution_query = load_query(distribution_path)
# random_query = load_query(random_path)

# training_set_percentage = 0.5
# Td = int(len(distribution_query) * training_set_percentage)
# Tr = int(len(random_query) * training_set_percentage)

# training_set = np.concatenate((distribution_query[0:Td], random_query[0:Tr]), axis=0)
# testing_set = np.concatenate((distribution_query[Td:], random_query[Tr:]), axis=0)

problem_type = 2
query_path = '/home/centos/Queryset/'
# scale 100
# training_set = np.genfromtxt(query_path+"prob"+str(problem_type)+"_train.csv", delimiter=',')
# testing_set = np.genfromtxt(query_path+"prob"+str(problem_type)+"_test.csv", delimiter=',')

# scale 50 and 10
training_set = np.genfromtxt(query_path+"prob"+str(problem_type)+"_train_scale"+str(scale_factor)+".csv", delimiter=',')
testing_set = np.genfromtxt(query_path+"prob"+str(problem_type)+"_test_scale"+str(scale_factor)+".csv", delimiter=',')

used_dims = [1,2,3,4]
num_dims = 16
column_names = ['_c'+str(i) for i in range(num_dims)]
column_name_dict = {}
for i in range(num_dims):
    column_name_dict[i] = column_names[i]

# hdfs_path_nora = 'hdfs://localhost:9000/user/cloudray/NORA/merged/'
# hdfs_path_qdtree = 'hdfs://localhost:9000/user/cloudray/QdTree/merged/'

# scale 100
# hdfs_path_nora = 'hdfs://192.168.6.62:9000/user/cloudray/NORA/prob'+str(problem_type)+'/merged/'
# hdfs_path_qdtree = 'hdfs://192.168.6.62:9000/user/cloudray/QdTree/prob'+str(problem_type)+'/merged/'
# # hdfs_path_kdtree = 'hdfs://192.168.6.62:9000/user/cloudray/KDTree/prob'+str(problem_type)+'/merged/'
# hdfs_path_kdtree = 'hdfs://192.168.6.62:9000/user/cloudray/KDTree/prob'+str(1)+'/merged/'

# scale 50 and 10
hdfs_base_path = 'hdfs://192.168.6.62:9000/user/cloudray/'
hdfs_path_nora = hdfs_base_path + 'NORA/prob' + str(problem_type) + '/scale' + str(scale_factor) + "/merged/"
hdfs_path_qdtree = hdfs_base_path + 'QdTree/prob' + str(problem_type) + '/scale' + str(scale_factor) + "/merged/"
hdfs_path_kdtree = hdfs_base_path + 'KDTree/prob' + str(problem_type) + '/scale' + str(scale_factor) + "/merged/"

# hdfs_path_nora = 'hdfs://192.168.6.62:9000/user/cloudray/NORA/scale100/reorganized'
# hdfs_path_qdtree = 'hdfs://192.168.6.62:9000/user/cloudray/QdTree/scale100/reorganized'
# hdfs_path_kdtree = 'hdfs://192.168.6.62:9000/user/cloudray/KDTree/scale100/reorganized'

# newly added
querytype = 0 # 0: SELECT *;  2: SELECT variance(_c0)
partition_base_path = '/home/centos/PartitionLayout/'

# scale 100
# nora_partition_path = partition_base_path + 'prob' + str(problem_type) + '_nora'
# qdtree_partition_path = partition_base_path + 'prob' + str(problem_type) + '_qdtree'
# kdtree_partition_path = partition_base_path + 'prob' + str(problem_type) + '_kdtree'

# scale 50 and 10
nora_partition_path = partition_base_path + 'prob' + str(problem_type) + '_nora_scale' + str(scale_factor)
qdtree_partition_path = partition_base_path + 'prob' + str(problem_type) + '_qdtree_scale' + str(scale_factor)
kdtree_partition_path = partition_base_path + 'prob' + str(problem_type) + '_kdtree_scale' + str(scale_factor)

# partition_base_path = '/home/ubuntu/PartitionLayout/'
# nora_partition = partition_base_path + 'nora_partitions_' + str(scale_factor)

In [12]:
# test query
# notice, there should not be any white space between and two pids
# sql = 'SELECT * FROM parquet.`hdfs://10.88.88.103:9000/user/cloudray/NORA/scale100/merged/partition_{164,165}.parquet`'
# sql = 'SELECT variance(_c0) FROM parquet.`hdfs://10.88.88.103:9000/user/cloudray/NORA/scale100/merged/partition_{164,165}.parquet`'
# result = spark.sql(sql).collect()

In [13]:
# len(result) # 0 and 1: 3124568
# len(result) # 0: 1556604
# len(result) # 1: 1567964

In [14]:
# NORA
# batch_query(testing_set, used_dims, column_name_dict, hdfs_path_nora, partition_index)

# SELECT *
# total query response time:  861.4990439414978
# average query response time:  17.229980878829956

# SELECT COUNT(*) # the advantage is more obvious when io of query result do not dominate the query time
# total query response time:  24.595819234848022
# average query response time:  0.4919163846969605

# SELECT variance(_c0)
# total query response time:  32.315288066864014
# average query response time:  0.6463057613372802

In [15]:
# Qd-Tree
# batch_query(testing_set, used_dims, column_name_dict, hdfs_path_qdtree)

# SELECT *
# total query response time:  1169.1192693710327
# average query response time:  23.382385387420655/

# SELECT COUNT(*)
# total query response time:  85.07339429855347
# average query response time:  1.7014678859710692

# SELECT variance(_c0)
# total query response time:  102.03884530067444
# average query response time:  2.040776906013489

In [22]:
# spark.catalog.listDatabases() # [Database(name='default', description='default database', locationUri='file:/home/centos/NORA_SPARK/spark-warehouse')]
# spark.catalog.listTables() # []
# spark.catalog.listTables('default') # []
# spark.catalog.refreshByPath('hdfs://192.168.6.62:9000/user/cloudray/QdTree/scale100/')

In [13]:
# Problem 1
# NORA
# batch_query(training_set, used_dims, column_name_dict, hdfs_path_nora, querytype, nora_partition_path)

In [14]:
# Problem 1
# Qd-Tree
# batch_query(training_set, used_dims, column_name_dict, hdfs_path_qdtree, querytype, qdtree_partition_path)

In [15]:
# Problem 1
# KDTree
# batch_query(training_set, used_dims, column_name_dict, hdfs_path_kdtree, querytype, kdtree_partition_path)

In [15]:
# Problem 2
# NORA
batch_query(testing_set, used_dims, column_name_dict, hdfs_path_nora, querytype, nora_partition_path)

pids: [41]
finish query 0
pids: [41]
finish query 1
pids: [41]
finish query 2
pids: [41]
finish query 3
pids: [52]
finish query 4
pids: [52]
finish query 5
pids: [52]
finish query 6
pids: [52]
finish query 7
pids: [26]
finish query 8
pids: [26]
finish query 9
pids: [26]
finish query 10
pids: [26]
finish query 11
pids: [26]
finish query 12
pids: [15]
finish query 13
pids: [15]
finish query 14
pids: [15]
finish query 15
pids: [15]
finish query 16
pids: [16]
finish query 17
pids: [49]
finish query 18
pids: [49]
finish query 19
pids: [49]
finish query 20
pids: [37]
finish query 21
pids: [37]
finish query 22
pids: [37]
finish query 23
pids: [37]
finish query 24
pids: [37]
finish query 25
pids: [31]
finish query 26
pids: [31]
finish query 27
pids: [38]
finish query 28
pids: [38]
finish query 29
pids: [38]
finish query 30
pids: [38]
finish query 31
pids: [42]
finish query 32
pids: [42]
finish query 33
pids: [48]
finish query 34
pids: [48]
finish query 35
pids: [48]
finish query 36
pids: [48]


In [ ]:
# Problem 2
# Qd-Tree
batch_query(testing_set, used_dims, column_name_dict, hdfs_path_qdtree, querytype, qdtree_partition_path)

pids: [55, 56, 40]
finish query 0
pids: [55, 56, 40]
finish query 1
pids: [55, 56, 40]
finish query 2
pids: [55, 56, 40]
finish query 3
pids: [47, 48]
finish query 4
pids: [47, 48]
finish query 5
pids: [47, 48]
finish query 6
pids: [47, 48]
finish query 7
pids: [33, 18, 38, 25]
finish query 8
pids: [33, 18, 38, 25]
finish query 9
pids: [33, 18, 38, 25]
finish query 10
pids: [33, 18, 38, 25]
finish query 11
pids: [33, 18, 38, 25]
finish query 12
pids: [15, 53, 54]
finish query 13
pids: [15, 53, 54]
finish query 14
pids: [15, 53, 54]
finish query 15
pids: [15, 53, 54]
finish query 16
pids: [54, 32]
finish query 17
pids: [45, 46]
finish query 18
pids: [45, 46]
finish query 19
pids: [45, 46]
finish query 20
pids: [15, 35]
finish query 21
pids: [15, 35]
finish query 22
pids: [15, 35]
finish query 23
pids: [35]
finish query 24
pids: [35]
finish query 25
pids: [30]
finish query 26
pids: [30]
finish query 27
pids: [36]
finish query 28
pids: [36]
finish query 29
pids: [36]
finish query 30
pids:

In [13]:
# Problem 2
# KDTree
batch_query(testing_set, used_dims, column_name_dict, hdfs_path_kdtree, querytype, kdtree_partition_path)

pids: [50]
finish query 0
pids: [50]
finish query 1
pids: [50]
finish query 2
pids: [50]
finish query 3
pids: [79]
finish query 4
pids: [79]
finish query 5
pids: [79]
finish query 6
pids: [79]
finish query 7
pids: [56]
finish query 8
pids: [56]
finish query 9
pids: [56]
finish query 10
pids: [56]
finish query 11
pids: [56]
finish query 12
pids: [12, 49]
finish query 13
pids: [12, 49]
finish query 14
pids: [12, 49]
finish query 15
pids: [12, 49]
finish query 16
pids: [11]
finish query 17
pids: [64]
finish query 18
pids: [64]
finish query 19
pids: [64]
finish query 20
pids: [69]
finish query 21
pids: [69]
finish query 22
pids: [69]
finish query 23
pids: [69]
finish query 24
pids: [69]
finish query 25
pids: [83]
finish query 26
pids: [83]
finish query 27
pids: [70]
finish query 28
pids: [70]
finish query 29
pids: [70]
finish query 30
pids: [70]
finish query 31
pids: [49, 50]
finish query 32
pids: [49, 50]
finish query 33
pids: [56, 58]
finish query 34
pids: [56, 58]
finish query 35
pids: 

In [19]:
# check number of row groups
import pyarrow as pa
import pyarrow.parquet as pq
# fs = pa.hdfs.connect()
fs = pa.fs.HadoopFileSystem('192.168.6.62', port=9000, user='hdfs', replication=1)

In [30]:
path1 = 'hdfs://192.168.6.62:9000/user/cloudray/NORA/prob1/merged/partition_94.parquet'
path2 = 'hdfs://192.168.6.62:9000/user/cloudray/KDTree/prob1/merged/partition_98.parquet'
# path1 = 'hdfs://192.168.6.62:9000/user/cloudray/NORA/scale100/partition_99.parquet'             # 51 row groups, serialized_size: 86891
# path2 = 'hdfs://192.168.6.62:9000/user/cloudray/NORA/scale100/reorganized/partition_99.parquet'   # 1 rouw group, serialized_size: 7872
# path1 = 'hdfs://192.168.6.62:9000/user/cloudray/QdTree/scale100/partition_0.parquet'            # 51 row groups, serialized_size: 90136
# path2 = 'hdfs://192.168.6.62:9000/user/cloudray/QdTree/scale100/reorganized/partition_0.parquet'  # 1 rouw group, serialized_size: 9117
# path1 = 'hdfs://192.168.6.62:9000/user/cloudray/KDTree/scale100/partition_99.parquet'           # 51 row groups, serialized_size: 86183
# path2 = 'hdfs://192.168.6.62:9000/user/cloudray/KDTree/scale100/reorganized/partition_99.parquet' # 1 rouw group, serialized_size: 9053
# serialized_size should be the size of footer data

In [32]:
fw1 = fs.open_input_file(path1)
meta1 = pa.parquet.read_metadata(fw1, memory_map=False)
print(meta1)
print(meta1.row_group(0))
fw1.close()

fw2 = fs.open_input_file(path2)
meta2 = pa.parquet.read_metadata(fw2, memory_map=False)
print(meta2)
print(meta2.row_group(0))
fw2.close()

  created_by: parquet-cpp version 1.5.1-SNAPSHOT
  num_columns: 16
  num_rows: 22730171
  num_row_groups: 1
  format_version: 1.0
  serialized_size: 9138
  num_columns: 16
  num_rows: 22730171
  total_byte_size: 905159169
  created_by: parquet-cpp version 1.5.1-SNAPSHOT
  num_columns: 16
  num_rows: 1280845
  num_row_groups: 1
  format_version: 1.0
  serialized_size: 9053
  num_columns: 16
  num_rows: 1280845
  total_byte_size: 52271062
